# Prep

Setting up some prior functionality

In [2]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

2.6.0+cu126 True


# Load a model

First we have to decide if our model should be pretrained. 

This greatly depends on the size of a dataset. Smaller datasets rely more on finetuning. 

In [3]:
pretrained = False

if pretrained:
    # Get pretrained weights
    checkpoint = torch.hub.load_state_dict_from_url(
                url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
                map_location='cpu',
                check_hash=True)
    # Remove class weights
    del checkpoint["model"]["class_embed.weight"]
    del checkpoint["model"]["class_embed.bias"]

    # SaveOGH
    torch.save(checkpoint,
               'detr-r50_no-class-head.pth')

# Dataset

Our dataset should be loadable as a COCO format

This allows us to use the pycocotools to load the data dict for the main python script

In [4]:
dataset_file = "coco" # alternatively, implement your own coco-type dataset loader in datasets and add this "key" to datasets/__init__.py

dataDir='datasets/sentinel2_coco' 
# dataDir='datasets/ship_playground_coco' 
num_classes = 2 # this int should be the actual number of classes + 1 (for no class)

outDir = 'outputs'
resume = "detr-r50_no-class-head.pth" if pretrained else "scratch"
frozen_weights = None

# Training

We use the main.py script to run our training

In [5]:
!python main.py \
  --dataset_file $dataset_file \
  --coco_path $dataDir \
  --output_dir $outDir \
  --resume $resume \
  --num_classes $num_classes \
  --lr 1e-4 \
  --lr_backbone 1e-5 \
  --epochs 80

^C


# Results

Quick and easy overview of the training results

In [5]:
from util.plot_utils import plot_logs

from pathlib import Path

log_directory = [Path(outDir)]

In [6]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

-> missing log.txt.  Have you gotten to Epoch 1 in training?
--> full path of missing log file: outputs\log.txt


In [7]:
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
    )

plot_logs(log_directory,
          fields_of_interest)

-> missing log.txt.  Have you gotten to Epoch 1 in training?
--> full path of missing log file: outputs\log.txt


In [8]:


fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)   

# from util.plot_utils import plot_precision_recall
# plot_precision_recall(log_directory)

-> missing log.txt.  Have you gotten to Epoch 1 in training?
--> full path of missing log file: outputs\log.txt
